In [ ]:
pip install langchain google-search-results serpapi openai pinecone-client unstructured -qU langchain-openai -U langchain-community

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

## Fetching and processing articles

Fetchin news articles related to a company and converting them into chunks

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.agents import initialize_agent, Tool, AgentType

import os

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = "DTN1wiFDfM0FczleBcGBCVkOy7Clf6mEsRDf7KRDXNDcqUItGTKqJQQJ99ALACHYHv6XJ3w3AAAAACOGIxqq"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://delvi-m4wlt8x5-eastus2.cognitiveservices.azure.com/'
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = 'gpt-35-turbo-16k'
os.environ["AZURE_OPENAI_API_VERSION"] = '2024-02-01'
os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"] = "text-embedding-ada-002"



from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [ ]:
from serpapi import GoogleSearch

api_key = 'b5d665bfd0fd4faca880d2e76ba14407f1640b71cc428b68a5d001d25af45e98'

def get_articles_from_serpapi(query):
    search = GoogleSearch({
        "q": query,
        "api_key": api_key
    })
    results = search.get_dict()

    articles = []
    snippets = []
    for result in results.get('related_questions', []):
        articles.append(result['link'])  # Collect the URLs of the articles
        snippets.append(result['snippet'])  # Collect the snippets of the articles

    if len(articles) < 5:
      return articles,snippets
    else:
      return articles[:5],snippets[:5]


In [ ]:
query = "Recent financial news of google shares"
urls,snippets = get_articles_from_serpapi(query)

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def url_document_splitting(url,data):
  loader = UnstructuredURLLoader(urls=[url])
  documents = loader.load()

  r_splitter = RecursiveCharacterTextSplitter(
      separators=["\n\n", "\n", " "],
      chunk_size=200,
      chunk_overlap=0,
  )

  docs = r_splitter.split_documents(documents)
  chunks = []
  for items in docs:
    chunks.append(items.page_content)

  data = data + chunks
  return data

data = []
for url in urls:
  data = url_document_splitting(url,data)

data = data + snippets

## Embedding and VectorDB

Embed the chunks above and store them in pinecone

In [ ]:
import pinecone

os.environ["PINECONE_API_KEY"] = "pcsk_4CgvP2_KTuQTRYwTkxjKvYg9wzNZWNG1vt7yUyvVU3QDALxxAgptFm2EHbkXatMFbYu2Je"
pinecone_api_key = os.environ["PINECONE_API_KEY"]
ps = pinecone.Pinecone(api_key = pinecone_api_key )

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint= os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key= os.environ["AZURE_OPENAI_API_KEY"],
    api_version = os.environ["AZURE_OPENAI_API_VERSION"]
)


In [ ]:
import pandas as pd

df = pd.DataFrame(data,columns=['text'])
df = df.reset_index()

def embedding_function(text):
    response = client.embeddings.create(
        input = text,
        model= "text-embedding-ada-002"
    )

    return response.data[0].embedding

df["embeddings"] = df["text"].apply(embedding_function)

In [ ]:
from pinecone import ServerlessSpec

pinecone_index_name = "finance-finder"

# ps.create_index(
#     name=pinecone_index_name,
#     dimension=1536,
#     metric="cosine",
#     spec=ServerlessSpec(
#       cloud="aws",
#       region="us-east-1"
#     )
# )

In [ ]:
pinecone_index = ps.Index(pinecone_index_name)

In [ ]:
# from tqdm.auto import tqdm

# batch_size = 1000

# for i in tqdm(range(0, len(df["text"]), batch_size)):
#     i_end = min(i + batch_size, len(df["text"]))
#     ids = [str(x) for x in range(i, i_end)]
#     metadata = [{"text": text} for text in df["text"][i:i_end]]
#     embedding_text_batch = df["embeddings"][i:i_end].tolist()
#     records = zip(ids, embedding_text_batch, metadata)
#     pinecone_index.upsert(vectors=records)

# pinecone_index.describe_index_stats()

  0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'pinecone_index' is not defined

In [ ]:
query = "google"

user_query = embedding_function(query)

pinecone_answer = pinecone_index.query(
    vector=user_query,
    top_k=3,
    include_metadata=True
)

# Display the results
for result in pinecone_answer["matches"]:
    print(f"{round(result['score'], 2)}, {result['metadata']['text']}")
    print("-" * 120)

0.8, Stocks Online
------------------------------------------------------------------------------------------------------------------------
0.8, Snapshot
------------------------------------------------------------------------------------------------------------------------
0.79, GOOGL stock forecasts by analyst
------------------------------------------------------------------------------------------------------------------------


## AI agents to get finance data

In [ ]:
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper

os.environ["SERPAPI_API_KEY"] = "b5d665bfd0fd4faca880d2e76ba14407f1640b71cc428b68a5d001d25af45e98"
finance = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())

In [ ]:
print(finance.run("Tata motors"))


Query: Tata motors
stock: TATAMOTORS:NSE
price: ₹789.00
percentage: 3.13
movement: Up
us: price = 42732.13, movement = Up
europe: price = 19906.08, movement = Down
asia: price = 39894.54, movement = Down



In [ ]:
finance_tool = Tool(
    name = "Google Finance",
    func = finance.run,
    description = "Fetches the stock market metrics related to a particular company"
)

tools = [finance_tool]

agent = initialize_agent(
    tools, model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

<ipython-input-19-dc893594a783>:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
agent.run("What is the movement pattern of the Tata motors shares")

<ipython-input-22-60226612e79d>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is the movement pattern of the Tata motors shares")




> Entering new AgentExecutor chain...
I need to fetch the stock market metrics related to Tata Motors in order to determine its movement pattern.
Action: Google Finance
Action Input: 'Tata Motors'
Observation: 
Query: 'Tata Motors'
No summary information
us: price = 42732.13, movement = Up
europe: price = 19906.08, movement = Down
asia: price = 39894.54, movement = Down

Thought:I have fetched the stock market metrics related to Tata Motors. The movement pattern of Tata Motors shares is as follows:
- In the US market, the price has increased and the movement is up.
- In the European market, the price has decreased and the movement is down.
- In the Asian market, the price has decreased and the movement is down.

Final Answer: The movement pattern of Tata Motors shares is mixed, with an upward movement in the US market and downward movements in the European and Asian markets.

> Finished chain.


'The movement pattern of Tata Motors shares is mixed, with an upward movement in the US market and downward movements in the European and Asian markets.'